In [1]:
import os
import time
import random
from itertools import count, combinations

import chess
import pandas as pd
from tqdm import tqdm 

from IPython.display import clear_output, HTML

In [2]:
# rename all files in the "bots" directory to remove spaces and special characters
# remove all . before the extension
# this is necessary for the import to work

bots_dir = "bots"
for filename in os.listdir(bots_dir):
    if filename.endswith(".py"):
        new_filename = filename.replace(" ", "_") \
                                .replace(".", "_") \
                                .replace("-", "_") \
                                .replace("(", "_") \
                                .replace(")", "") \
                                .replace("__", "_") \
                                .replace("__", "_") \
        
        new_filename = new_filename[:-3] + ".py"
        os.rename(os.path.join(bots_dir, filename), os.path.join(bots_dir, new_filename))
        print(f"{filename} -> {new_filename}")

wrapper_M_Opat.py -> wrapper_M_Opat.py
bot_random.py -> bot_random.py


In [3]:
bot_dir = 'bots'
bot_files = [bot_file for bot_file in os.listdir(bot_dir) if bot_file.endswith('.py')]
bot_files = sorted(bot_files)

bot_files

['bot_random.py', 'wrapper_M_Opat.py']

In [10]:
# import bots from the files
from importlib import import_module, reload

bot_classes = []

for bot_file in bot_files:
    bot_name = bot_file[:-3]
    try:
        bot_module = reload(import_module(f"{bot_dir}.{bot_name}"))
    except Exception as e:
        print(f"Error importing bot {bot_name}: {e}")
        continue
    bot_class = getattr(bot_module, "ChessBot")
    bot_class.name = bot_name
    
    bot_classes.append(bot_class)

print(bot_classes)

Error importing bot wrapper_M_Opat: dlopen(/var/folders/w3/2rfcg8md39x8h72m0ssfy9n80000gn/T/tmp36hheu6o.so, 0x0006): tried: '/var/folders/w3/2rfcg8md39x8h72m0ssfy9n80000gn/T/tmp36hheu6o.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/var/folders/w3/2rfcg8md39x8h72m0ssfy9n80000gn/T/tmp36hheu6o.so' (no such file), '/var/folders/w3/2rfcg8md39x8h72m0ssfy9n80000gn/T/tmp36hheu6o.so' (not a mach-o file), '/private/var/folders/w3/2rfcg8md39x8h72m0ssfy9n80000gn/T/tmp36hheu6o.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/private/var/folders/w3/2rfcg8md39x8h72m0ssfy9n80000gn/T/tmp36hheu6o.so' (no such file), '/private/var/folders/w3/2rfcg8md39x8h72m0ssfy9n80000gn/T/tmp36hheu6o.so' (not a mach-o file)
[<class 'bots.bot_random.ChessBot'>]


In [5]:
class Judge():
    def __init__(self, player_1, player_2, time_limit=300): # 5 minutes per player
        self.player_1 = player_1            # white
        self.player_2 = player_2            # black
        self.time_limit = time_limit

    def run_game(self, initial_board_fen: str = None, slow_down: bool = False, verbose: bool = False):
        if initial_board_fen is None:
            initial_board_fen = chess.Board().fen()
        board = chess.Board(initial_board_fen)

        player_times = [0, 0]
        history = []
        notes = []
        winner = None

        for i in count(0, 1):
            history.append(board.fen())

            if board.is_checkmate():
                winner = "black" if i%2 == 0 else "white"
                notes.append(f"Checkmate! Winner is {winner}")
                break
            elif board.is_stalemate():
                notes.append("Stalemate! It's a draw.")
                break
            elif board.is_insufficient_material():
                notes.append("Insufficient material! It's a draw.")
                break
            elif board.is_seventyfive_moves():
                notes.append("Seventy-five moves without a capture or pawn move! It's a draw.")
                break
            elif board.is_fivefold_repetition():
                notes.append("Fivefold repetition! It's a draw.")
                break
            elif board.is_game_over():
                notes.append("Game over! It's a draw.")
                break

            board_fen = board.fen()

            start = time.time()
            try:
                if i % 2 == 0:
                    move = self.player_1(board_fen)
                else:
                    move = self.player_2(board_fen)
            except Exception as e:
                winner = "black" if i%2 == 0 else "white"
                notes.append(f"Exception: {e}. Winner is {winner}")
                break
            end = time.time()

            player_times[i%2] += end - start

            if player_times[i%2] > self.time_limit:
                winner = "black" if i%2 == 0 else "white"
                notes.append(f"\nTime limit exceeded! Winner is {winner}")
                break

            if not board.is_legal(move):
                winner = "black" if i%2 == 0 else "white"
                hallucinator = "white" if i%2 == 0 else "black"
                notes.append(f"Illegal board move. The {hallucinator} is hallucinating.")
                break

            board.push(move)

            if verbose: 
                clear_output(wait=True)
                display(board)
                display(HTML(f"{self.player_1.name} (white) vs {self.player_2.name} (black)"))

            # slow down the bots so that we can see them
            if slow_down: 
                time.sleep(.001)

        results = {}
        results['winner'] = winner
        results['times'] = player_times
        results["notes"] = notes
        results["history"] = history

        return results

In [6]:
with open('opennings.txt', 'r') as file:
    opennings = file.read().split('\n')

In [7]:
# play all bots against each other

N_GAMES = 1 # per pair color (so N_GAMES * 2 games per pair of bots)

today = time.strftime("%Y-%m-%d")
today_results = f"results/{today}_results.csv"

columns=[
    'white_bot', 'black_bot', 'winner', 'white_time', 'black_time', 'notes', 'history'
]

combin = combinations(bot_classes, 2)
combin = list(combin)
random.shuffle(combin)
combin = tqdm(combin)

for bot1, bot2 in combin:
    for white, black in [(bot1, bot2), (bot2, bot1)]:
        if os.path.exists(today_results) and pd.read_csv(today_results).query(
            f"(white_bot == '{white.name}') & (black_bot == '{black.name}')"
        ).shape[0] >= N_GAMES:
            continue

        for i in range(N_GAMES):
            judge = Judge(white(), black())
            random_opening = random.choice(opennings)
            results = judge.run_game(random_opening, verbose=True)

            game_results = pd.DataFrame([[
                white.name, black.name, results['winner'], 
                results['times'][0], results['times'][1], 
                "; ".join(results['notes']),
                "\t".join(results['history'])
            ]], columns=columns)

            game_results.to_csv(
                today_results, 
                mode='a', 
                index=False,
                header=not os.path.exists(today_results), 
            )

clear_output(wait=True)

0it [00:00, ?it/s]


In [8]:
game_results = pd.read_csv(today_results)

# make sure the bots are in bot list
game_results = game_results[
    game_results['white_bot'].isin([bot.name for bot in bot_classes]) &
    game_results['black_bot'].isin([bot.name for bot in bot_classes])
]

leaderboard = pd.DataFrame(columns=['bot', 'wins', 'draws', 'losses', 'avg_time', 'score'])

for bot in bot_classes:
    bot_name = bot.name

    bot_wins = sum((game_results["white_bot"] == bot_name) & (game_results["winner"] == "white")) + \
               sum((game_results["black_bot"] == bot_name) & (game_results["winner"] == "black"))
    
    bot_draws = sum((game_results["white_bot"] == bot_name) & (game_results["winner"].isna())) + \
                sum((game_results["black_bot"] == bot_name) & (game_results["winner"].isna()))
    
    bot_losses = sum((game_results["white_bot"] == bot_name) & (game_results["winner"] == "black")) + \
                    sum((game_results["black_bot"] == bot_name) & (game_results["winner"] == "white"))
    
    avg_bot_time = (game_results[(game_results["white_bot"] == bot_name)]["white_time"].sum() + \
                game_results[(game_results["black_bot"] == bot_name)]["black_time"].sum()) / \
                    (bot_wins + bot_draws + bot_losses)
    
    bot_score = bot_wins + 0.5 * bot_draws + 0 * bot_losses

    leaderboard.loc[len(leaderboard)] = [bot_name, bot_wins, bot_draws, bot_losses, avg_bot_time, bot_score]

leaderboard = leaderboard.sort_values(by='score', ascending=False)
leaderboard = leaderboard.reset_index(drop=True)

leaderboard

FileNotFoundError: [Errno 2] No such file or directory: 'results/2024-02-26_results.csv'

In [ ]:
today_leaderboard = f"results/{today}_leaderboard.csv"

game_results.to_csv(today_results, index=False)
leaderboard.to_csv(today_leaderboard, index=False)

In [ ]:
print(leaderboard.to_markdown())